In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !apt install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
!pip install datasets transformers torch torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 771.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

<br><br> &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E;
<br><br>

<h5><b>§ Version 6</b></h5>

In [10]:
source_dir = "/content/drive/MyDrive/Audio/ffmpeg_source/"
process_dir = "/content/drive/MyDrive/Audio/ffmpeg_process/"
product_dir = "/content/drive/MyDrive/Audio/ffmpeg_product/"
match_filename = "match_file.txt"
close_filename = "high_english.txt"
silence = "silence_4s.mp3"

# language = "French"
lang1 = 'fra'
lang2 = 'eng'

In [4]:
import torch
# import gradio as gr
import numpy as np
# import matplotlib.pyplot as plt
# import librosa.display
import os
import subprocess
import re
import time
# import tqdm
from datasets import load_dataset, Dataset, Audio
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline

In [ ]:
import os
import subprocess
import re
# import tqdm

### Slices audio into sentences. ###

# Collects names of all files to be processed.
source_list = [ j for j in os.listdir(source_dir) if not j.startswith('.') ]
# print(source_list)

for source in source_list:
    # Change format to .mp3
    m4a_to_mp3_cmd = f"ffmpeg -i {source_dir}{source} {process_dir}{os.path.splitext(source)[0]}.mp3"
    m4a_to_mp3_split = m4a_to_mp3_cmd.split()
    result1 = subprocess.run(m4a_to_mp3_split, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # print(result1.stderr)

    # Create list of silences between audio clips.
    detect_cmd = f"ffmpeg -i {process_dir}{os.path.splitext(source)[0]}.mp3 -af silencedetect=noise=0.0001 -hide_banner -f null - "
    detect_split = detect_cmd.split()
    result2 = subprocess.run(detect_split, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # print(result2.stderr)

    start_silence = re.findall(r"silence_start: (.*)", result2.stderr)
    end_silence = re.findall(r"silence_end: (\d*\.\d*)", result2.stderr)
    start_times = end_silence[:-1]
    end_times = start_silence[1:]
    # print(start_times)

    ### Splits the audio into clips ###
    for i in range(len(start_times)):
    # for i in range(5):
        product_name = f"{product_dir}{os.path.splitext(source)[0]}_{i}.mp3"
        # product_name = f"{product_dir}/{source}_{i}.m4a"
        clip_cmd = f'ffmpeg -ss {start_times[i]} -i {process_dir}{os.path.splitext(source)[0]}.mp3 -to {end_times[i]} -c copy -hide_banner -bitexact -f mp3 -copyts -avoid_negative_ts 2 {product_name}'
        # print(f"{source_dir}{os.path.splitext(source)[0]}.mp3")
        clip_split = clip_cmd.split()
        result3 = subprocess.run(clip_split, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        # print(result3.stderr)

In [5]:
from datasets import Dataset, Audio
import os

### Creates Dataset from generated audio files. ###

# Filepath for clips producted from source audio.
product_list = [ product_dir + j for j in os.listdir(product_dir) if not j.startswith('.') ]
# print(product_list)

audio_dataset = Dataset.from_dict({"audio": product_list}).cast_column("audio", Audio(sampling_rate=16_000))
# dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation[:10]")


print(audio_dataset[0]["audio"]["path"])
# sample_clip = next(iter(audio_dataset))



/content/drive/MyDrive/Audio/ffmpeg_product/Lesson_22_0.mp3


In [6]:
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from datasets import load_dataset
import time


# pipe = pipeline(model="sanchit-gandhi/whisper-medium-fleurs-lang-id", device=0)
pipe = pipeline(task="audio-classification", model="facebook/mms-lid-256", device=0)
# pipe.save_pretrained("/content/drive/MyDrive/Audio/hf_models/mms-lid-256_v2")

# pipe = pipeline.from_pretrained(model="/content/drive/MyDrive/Audio/hf_models/mms-lid-256", device=0)
# pipe = pipeline(task="audio-classification", model="/content/drive/MyDrive/Audio/hf_models/mms-lid-256")


infer_list = []
path_list = []
# for sample in audio_dataset:
start_time = time.time()
# KeyDataset is a util that will just output the item we're interested in.
for track in KeyDataset(audio_dataset, "audio"):
    inference = pipe(track)
    # print(inference)
    infer_list.append(inference)
    path_list.append(track["path"])
end_time = time.time()
print(f"{(end_time - start_time)//60}m {round((end_time - start_time)%60, 3)}s")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/6.61k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


0.0m 45.962s


In [ ]:

match_list = []
high_english = []
# Selection algo #2
for j in range(len(infer_list)):
    for id_score in infer_list[j]:
        if id_score["label"] == lang1:
            lang1_score = id_score
        elif id_score["label"] == lang2:
            lang2_score = id_score
    if lang1_score["score"] > 0.50:
        match_list.append("file " + "'" + path_list[j] + "'\n")
        # match_list.append("file " + "'" + f"{process_dir}{silence}" + "'\n")
    elif lang2_score["score"] > 0.50:
        high_english.append("file " + "'" + path_list[j] + "'\n")

match_list.sort(key=lambda s: int(re.findall(r'\d+', s)[1]))
high_english.sort(key=lambda s: int(re.findall(r'\d+', s)[1]))
spaced_matches = []
for item in match_list:
    spaced_matches.append(item)
    spaced_matches.append("file " + "'" + f"{process_dir}{silence}" + "'\n")

# print(match_list)
# print(high_english)
with open(f"{process_dir}{match_filename}", 'w') as match_file:
    with open(f"{process_dir}{close_filename}", 'w') as english_file:
        [ match_file.write(track_path) for track_path in spaced_matches ]
        [ english_file.write(track_path2) for track_path2 in high_english ]


In [ ]:
import subprocess
import os

# for use inside the 'for source in source_list' loop:
# concat_cmd = f"ffmpeg -f concat -safe 0 -i {product_dir}{match_filename} -c copy {process_dir}{os.path.splitext(source)[0]}_{lang1}.mp3"

concat_cmd = f"ffmpeg -f concat -safe 0 -i {process_dir}{match_filename} -c copy {process_dir}Lesson_22_{lang1}.mp3"

concat_split = concat_cmd.split()
result4 = subprocess.run(concat_split, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# print(result4.stderr)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

<br><br> &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E; &#x1F47E;<br><br>

In [ ]:
# from transformers import pipeline
# pipe2 = pipeline(model="facebook/mms-lid-256", device=0)
# pipe2.save_pretrained("/content/drive/MyDrive/Audio/hf_models/mms-lid-256", from_pt=True)